<a href="https://colab.research.google.com/github/hadesyi/Pair_Extraordinate/blob/main/Groq_API_LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Groq 속도 체험하기

**Created by: [Hyungho Byun](https://www.linkedin.com/in/hyungho-byun-6b2588224/)**
<br><br>
<a href="https://colab.research.google.com/drive/1boaH7FMRUD3xE2rpmWvJMwVue1GcoUWi?usp=sharing" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Groq의 빠른 추론 엔진을 LangChain에서 체험하는 코드를 소개합니다.    
langchain_groq를 통해 쉽게 연결할 수 있습니다.

In [ ]:
!pip install langchain langchain_groq langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.6/315.6 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.3 MB/s eta 0:00:00


현재 사용 가능한 모델과 API 제한은 아래와 같습니다.

| **ID**             | **REQUESTS PER MINUTE** | **REQUESTS PER DAY** | **TOKENS PER MINUTE** |
|--------------------|-------------------------|----------------------|-----------------------|
| **llama3-8b-8192** | 30                      | 14,400               | 30,000                |
| **gemma-7b-it**    | 30                      | 14,400               | 15,000                |
| **mixtral-8×7b-32768**| 30                   | 14,400               | 5,000                 |
| **llama3-70b-8192**| 30                      | 14,400               | 6,000                 |


In [ ]:

from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq # Groq-LangChain 연결

from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser

from langchain_community.callbacks import get_openai_callback # 토큰 카운트를 위한 콜백
import time

간단한 체인을 만들고 실행합니다.

https://console.groq.com/keys 에서 키를 생성해야 합니다.

In [ ]:

chat = ChatGroq(
    temperature=0,
    model="llama3-70b-8192",
    api_key="여기에 키 입력",
)

url = "https://wow.groq.com/why-groq/"
loader = WebBaseLoader(url)

docs = loader.load()
question = "Groq 엔진의 LLM 추론 속도가 빠른 이유는 무엇인가요? 한국어로 답변하세요."


system = "Answer the question from given contexts. Answer in Korean."
human = """
Context: {context}

---

Question: {question}
"""

prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

chain = prompt | chat | StrOutputParser()


start = time.time()
with get_openai_callback() as cb: # 토큰 개수를 세기 위한 콜백
    result = (chain.invoke(
            {"context":docs[0].page_content,
             'question':question}))
    print(result)
    print("---")

print()

print(f"Total Tokens: {cb.total_tokens}")
print(f"Prompt Tokens: {cb.prompt_tokens}")
print(f"Completion Tokens: {cb.completion_tokens}")

end = time.time()
elapsed_time = end - start
print(f"Elapsed Time: {elapsed_time} seconds")

Groq의 LPU(Inference Engine)는 LLM(Large Language Models) 추론 속도가 빠른 이유는 두 가지 병목 현상을 해결했기 때문입니다. 첫째, LPU는 컴퓨트 밀도가 높아 LLM에 대한 컴퓨트 용량이 증가합니다. 둘째, LPU는 외부 메모리 병목 현상을 제거하여 LLM 추론 속도를 크게 향상시켰습니다. 이러한 기술로 인해 LPU는 GPU보다 LLM 추론 속도가 훨씬 빠릅니다.
---

Total Tokens: 2383
Prompt Tokens: 2264
Completion Tokens: 119
Elapsed Time: 4.633583307266235 seconds
